In [1]:
import re, os
import pathlib as pl
import sys

In [2]:
#set home and allow for local imports
home = pl.Path(os.getcwd()).parent
sys.path.append(str(home))
from src.core import *

In [3]:
#set variables
inputs = home/'data'/'input'
outputs = home/'data'/'output'
gage_table = pd.read_excel(inputs/'gages.xlsx', 'BasinChar_all_HUCs_Final')

## Bulletin 17C Result Extraction

In [5]:
#get list of available reports
rpts = glob.glob(str(inputs)+'/**/*.rpt', recursive=True)

In [6]:
#assign link to gage record
gage_table['17C_link_rpt'] = gage_table['GID'].apply(lambda x: find_rpt_link(x,rpts))

In [7]:
gage_table

,OBJECTID,HUC8,GID,Station_Name,TDA_SqMi,NCDA_SqMi,CDA_SqMi,BP_Mi,BL_Mi,BW_Mi,...,NOAA_2yr24hr_in,NOAA_10yr24hr_in,CN,KSAT_mimps,USFWS_Pct_Wetlands,USFWS_Pct_LakePond,MeanEVI,MeanNDVI,B17C,17C_link_rpt
0,1,10120105,6382200,"PRITCHARD DRAW NEAR LANCE CREEK, WY",5.295270,0.00000,5.295270,12.112100,3.335270,1.587659,...,-9.999,-9.999,82,2.527000,0.109145,0.111923,0.185806,0.322918,Completed,c:\git\flow-regression-analysis\data\input\dss...
1,3,10120107,6394450,"PASS CREEK NEAR DEWEY, SD",35.898300,0.00000,35.898300,41.259701,10.862800,3.304701,...,1.856,2.757,73,4.239000,0.074669,0.008906,0.234554,0.455544,Duplicate entries with different results,c:\git\flow-regression-analysis\data\input\dss...
2,4,10120108,6396490,"WARBONNET CREEK NEAR HARRISON, NEBR.",24.062500,0.00000,24.062500,26.062599,6.221530,3.867618,...,1.901,2.838,74,10.732000,0.176870,0.239907,0.299597,0.508000,Completed,c:\git\flow-regression-analysis\data\input\dss...
3,5,10120108,6399700,PINEY CREEK NEAR ARDMORE SD,7.351190,0.00000,7.351190,13.474000,3.604290,2.039567,...,1.911,2.876,88,0.695000,0.234759,0.257339,0.215091,0.375855,Completed,c:\git\flow-regression-analysis\data\input\dss...
4,2,10120106,6400875,"HORSEHEAD CR AT OELRICHS,SD",188.076004,0.00000,188.076004,93.463699,21.193800,8.874104,...,2.013,3.024,86,1.044000,1.150819,0.941879,0.254542,0.429945,Completed,c:\git\flow-regression-analysis\data\input\dss...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
337,42,10150004,6461000,"MINNECHADUZA CREEK AT VALENTINE, NEBR.",393.828003,0.00000,393.828003,150.968002,34.248199,11.499232,...,2.266,3.243,68,63.226002,2.399545,0.115815,0.398990,0.602827,NaN,None
338,189,10230006,6610500,"Indian Creek at Council Bluffs, IA",6.935820,0.00000,6.935820,15.692100,4.546880,1.525402,...,2.986,4.335,68,8.869000,0.324772,1.332358,0.577800,0.777771,NaN,None
339,116,10210003,6780000,MIDDLE LOUP RIVER AT ROCKVILLE NEBR,5314.879883,412.11200,4902.770020,1003.619995,196.360992,24.968147,...,2.357,3.423,66,78.248001,1.813984,0.370883,0.358571,0.555053,Completed,None
340,141,10210008,6787500,"CALAMUS RIVER NEAR BURWELL, NEBR.",1008.119995,5.73762,1002.380005,376.446991,79.054100,12.679671,...,2.473,3.543,66,88.125999,5.897964,1.447431,0.355928,0.578167,NaN,None


In [8]:
gage_table.apply(lambda x: rpt_to_parquet(x['17C_link_rpt'],x['GID'],outputs/'17c'),axis = 1)

No frequency data pattern found in c:\git\flow-regression-analysis\data\input\dss\GagesForStantec\6382200\Bulletin17Results\Bulletin17Summary.rpt .skipping
There are different results results for the same gage at ['c:\\git\\flow-regression-analysis\\data\\input\\dss\\GagesForStantec\\6394450\\Bulletin17Results\\06394450\\06394450.rpt', 'c:\\git\\flow-regression-analysis\\data\\input\\dss\\GagesForStantec\\6394450\\Bulletin17Results\\06399450\\06399450.rpt']
There are different results results for the same gage at ['c:\\git\\flow-regression-analysis\\data\\input\\dss\\GagesForStantec\\6813700\\Bulletin17Results\\06813700\\06813700.rpt', 'c:\\git\\flow-regression-analysis\\data\\input\\dss\\GagesForStantec\\6813700\\Bulletin17Results\\6813700\\6813700.rpt']
There are different results results for the same gage at ['c:\\git\\flow-regression-analysis\\data\\input\\dss\\NemahaCounty\\Gage_Analysis_1706822549143\\Simulations\\Nemaha_County\\Bulletin17Results\\06814000-_Station\\06814000-_Sta

AssertionError: columns and data don't align for c:\git\flow-regression-analysis\data\input\dss\GagesForStantec\06834500\Bulletin17Results\06834500\06834500.rpt

In [ ]:
gage_table

,OBJECTID,HUC8,GID,Station_Name,TDA_SqMi,NCDA_SqMi,CDA_SqMi,BP_Mi,BL_Mi,BW_Mi,...,PRISM_yr_mm,NOAA_2yr24hr_in,NOAA_10yr24hr_in,CN,KSAT_mimps,USFWS_Pct_Wetlands,USFWS_Pct_LakePond,MeanEVI,MeanNDVI,B17C
0,1,10120105,6382200,"PRITCHARD DRAW NEAR LANCE CREEK, WY",5.295270,0.00000,5.295270,12.112100,3.335270,1.587659,...,361.872192,-9.999,-9.999,82,2.527000,0.109145,0.111923,0.185806,0.322918,Completed
1,3,10120107,6394450,"PASS CREEK NEAR DEWEY, SD",35.898300,0.00000,35.898300,41.259701,10.862800,3.304701,...,456.449524,1.856,2.757,73,4.239000,0.074669,0.008906,0.234554,0.455544,Need B17C
2,4,10120108,6396490,"WARBONNET CREEK NEAR HARRISON, NEBR.",24.062500,0.00000,24.062500,26.062599,6.221530,3.867618,...,434.512146,1.901,2.838,74,10.732000,0.176870,0.239907,0.299597,0.508000,Completed
3,5,10120108,6399700,PINEY CREEK NEAR ARDMORE SD,7.351190,0.00000,7.351190,13.474000,3.604290,2.039567,...,440.175690,1.911,2.876,88,0.695000,0.234759,0.257339,0.215091,0.375855,Completed
4,2,10120106,6400875,"HORSEHEAD CR AT OELRICHS,SD",188.076004,0.00000,188.076004,93.463699,21.193800,8.874104,...,452.369232,2.013,3.024,86,1.044000,1.150819,0.941879,0.254542,0.429945,Completed
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
337,42,10150004,6461000,"MINNECHADUZA CREEK AT VALENTINE, NEBR.",393.828003,0.00000,393.828003,150.968002,34.248199,11.499232,...,557.912537,2.266,3.243,68,63.226002,2.399545,0.115815,0.398990,0.602827,NaN
338,189,10230006,6610500,"Indian Creek at Council Bluffs, IA",6.935820,0.00000,6.935820,15.692100,4.546880,1.525402,...,840.832947,2.986,4.335,68,8.869000,0.324772,1.332358,0.577800,0.777771,NaN
339,116,10210003,6780000,MIDDLE LOUP RIVER AT ROCKVILLE NEBR,5314.879883,412.11200,4902.770020,1003.619995,196.360992,24.968147,...,580.326721,2.357,3.423,66,78.248001,1.813984,0.370883,0.358571,0.555053,Completed
340,141,10210008,6787500,"CALAMUS RIVER NEAR BURWELL, NEBR.",1008.119995,5.73762,1002.380005,376.446991,79.054100,12.679671,...,650.888550,2.473,3.543,66,88.125999,5.897964,1.447431,0.355928,0.578167,NaN
